# Titanic classification model

Using StatQuest video on decision forests: clustering for missing data, n optimization, etc

(video 1)[https://youtu.be/J4Wdy0Wc_xQ?si=L08xHb3JQvmHEHjj]

(video 2)[https://youtu.be/sQ870aTKqiM?si=tXDFxqsgornauspB]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

# Figures inline and set visualization style
%matplotlib inline
sns.set()

In [14]:
train_data = pd.read_csv("./train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data = pd.read_csv("./test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [15]:
summary = pd.DataFrame({
    'Column Type': train_data.dtypes,
    'Missing Values': train_data.isnull().sum(),
    'Total': train_data.count(),
    'Uniques': train_data.nunique(),
    'min': train_data.min(numeric_only=True),
    'mean': train_data.mean(numeric_only=True),
    'median': train_data.median(numeric_only=True),
    'max': train_data.max(numeric_only=True)
}).reset_index().rename(columns={'index': 'Column'})

print("Train data summary:\n")
display(summary)

Train data summary:



,Column,Column Type,Missing Values,Total,Uniques,min,mean,median,max
0,Age,float64,177,714,88,0.42,29.699118,28.0000,80.0000
1,Cabin,object,687,204,147,NaN,NaN,NaN,NaN
2,Embarked,object,2,889,3,NaN,NaN,NaN,NaN
3,Fare,float64,0,891,248,0.00,32.204208,14.4542,512.3292
4,Name,object,0,891,891,NaN,NaN,NaN,NaN
5,Parch,int64,0,891,7,0.00,0.381594,0.0000,6.0000
6,PassengerId,int64,0,891,891,1.00,446.000000,446.0000,891.0000
7,Pclass,int64,0,891,3,1.00,2.308642,3.0000,3.0000
8,Sex,object,0,891,2,NaN,NaN,NaN,NaN
9,SibSp,int64,0,891,7,0.00,0.523008,0.0000,8.0000


Based on above, filling Cabin nan's with an empty string 

In [ ]:
summary = pd.DataFrame({
    'Column Type': test_data.dtypes,
    'Missing Values': test_data.isnull().sum(),
    'Total': test_data.count(),
    'Uniques': test_data.nunique(),
    'min': test_data.min(numeric_only=True),
    'mean': test_data.mean(numeric_only=True),
    'median': test_data.median(numeric_only=True),
    'max': test_data.max(numeric_only=True)
}).reset_index().rename(columns={'index': 'Column'})

print("Test data summary:\n")
display(summary)

Test data summary:



,Column,Column Type,Missing Values,Total,Uniques,min,mean,max
0,Age,float64,86,332,79,0.17,30.272590,76.0000
1,Cabin,object,327,91,76,NaN,NaN,NaN
2,Embarked,object,0,418,3,NaN,NaN,NaN
3,Fare,float64,1,417,169,0.00,35.627188,512.3292
4,Name,object,0,418,418,NaN,NaN,NaN
5,Parch,int64,0,418,8,0.00,0.392344,9.0000
6,PassengerId,int64,0,418,418,892.00,1100.500000,1309.0000
7,Pclass,int64,0,418,3,1.00,2.265550,3.0000
8,Sex,object,0,418,2,NaN,NaN,NaN
9,SibSp,int64,0,418,7,0.00,0.447368,8.0000


### Preprocessing steps

In [21]:
train_data['Age'] = train_data.Age.fillna(train_data.Age.median())
train_data['Fare'] = train_data.Fare.fillna(train_data.Fare.median())
#train_data = pd.get_dummies(train_data, columns=['Sex'], drop_first=True) # avoiding this and using mapping to ensure consistency when dealing with tes data later on
train_data['Sex'] = train_data['Sex'].map({'female':0,'male':1})
train_data['Embarked'] = train_data['Embarked'].map({'C':0, 'Q':1, 'S':2})

In [18]:
summary = pd.DataFrame({
    'Column Type': train_data.dtypes,
    'Missing Values': train_data.isnull().sum(),
    'Total': train_data.count(),
    'Uniques': train_data.nunique(),
    'min': train_data.min(numeric_only=True),
    'median': train_data.median(numeric_only=True),
    'mean': train_data.mean(numeric_only=True),
    'max': train_data.max(numeric_only=True)
}).reset_index().rename(columns={'index': 'Column'})

print("Train data summary:\n")
display(summary)

Train data summary:



,Column,Column Type,Missing Values,Total,Uniques,min,median,mean,max
0,Age,float64,0,891,88,0.42,28.0000,29.361582,80.0000
1,Cabin,object,687,204,147,NaN,NaN,NaN,NaN
2,Embarked,object,2,889,3,NaN,NaN,NaN,NaN
3,Fare,float64,0,891,248,0.00,14.4542,32.204208,512.3292
4,Name,object,0,891,891,NaN,NaN,NaN,NaN
5,Parch,int64,0,891,7,0.00,0.0000,0.381594,6.0000
6,PassengerId,int64,0,891,891,1.00,446.0000,446.000000,891.0000
7,Pclass,int64,0,891,3,1.00,3.0000,2.308642,3.0000
8,Sex,int64,0,891,2,0.00,1.0000,0.647587,1.0000
9,SibSp,int64,0,891,7,0.00,0.0000,0.523008,8.0000


In [22]:
features = ['Age', 'Sex', 'Embarked', 'Fare']
rf = RandomForestClassifier()
rf.fit(train_data[features], train_data['Survived'])

RandomForestClassifier()

#### Preprocessing test data
Not using the test data itself for any nan replacements

In [23]:
test_data['Age'] = test_data.Age.fillna(28)
test_data['Fare'] = test_data.Fare.fillna(14.4542) # from training data
test_data['Sex'] = test_data['Sex'].map({'female':0,'male':1})
test_data['Embarked'] = test_data['Embarked'].map({'C':0, 'Q':1, 'S':2})

In [24]:
# model.label_classes() # run to confirm order for np.where()
y_pred = rf.predict(test_data[features])

In [25]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': y_pred})
output.to_csv('submission_sklearn_v0.csv', index=False)

## Results
Score: 0.61961